In [11]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.trainers import BpeTrainer

tokenizer = Tokenizer.from_pretrained("gpt2")

In [4]:
from tokenizers import Tokenizer

# Load Huggingface GPT-2 fast BPE tokenizer
tokenizer = Tokenizer.from_pretrained("gpt2")

# Read your full text
with open('out.txt', 'r') as f:
    text = f.read()

# Define chunk size (characters)
chunk_size = 10_000_000  # About ~10MB per chunk

# Split text into smaller manageable pieces
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Tokenize each chunk
tokens = []

print(f"Tokenizing {len(chunks)} chunks...")

for idx, chunk in enumerate(chunks):
    output = tokenizer.encode(chunk)
    tokens.extend(output.tokens)

    # Progress update
    if idx % 5 == 0:
        print(f"Chunk {idx+1}/{len(chunks)} done. Total tokens so far: {len(tokens)}")

print(f"Tokenization complete! Total tokens: {len(tokens)}")


c:\Github\Self-Made-Transformer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tokenizing 16 chunks...
Chunk 1/16 done. Total tokens so far: 2681563
Chunk 6/16 done. Total tokens so far: 16495997
Chunk 11/16 done. Total tokens so far: 29765540
Chunk 16/16 done. Total tokens so far: 42662204
Tokenization complete! Total tokens: 42662204


In [5]:
print(tokens[:100])
print(tokenizer.get_vocab_size())

['**', 'The', 'ĠProject', 'ĠGutenberg', 'ĠE', 'text', 'Ġof', 'Ġ20', ',', '000', 'ĠLe', 'agues', 'ĠUnder', 'Ġthe', 'ĠSea', '**', 'Ċ', '*****', 'This', 'Ġfile', 'Ġshould', 'Ġbe', 'Ġnamed', 'Ġ200', '0010', '.', 'txt', 'Ġor', 'Ġ200', '0010', '.', 'zip', '****', '**', 'ĊĊ', 'Ċ', 'Please', 'Ġtake', 'Ġa', 'Ġlook', 'Ġat', 'Ġthe', 'Ġimportant', 'Ġinformation', 'Ġin', 'Ġthis', 'Ġheader', '.', 'Ċ', 'We', 'Ġencourage', 'Ġyou', 'Ġto', 'Ġkeep', 'Ġthis', 'Ġfile', 'Ġon', 'Ġyour', 'Ġown', 'Ġdisk', ',', 'Ġkeeping', 'Ġan', 'Ċ', 'elect', 'ronic', 'Ġpath', 'Ġopen', 'Ġfor', 'Ġthe', 'Ġnext', 'Ġreaders', '.', 'Ġ', 'ĠDo', 'Ġnot', 'Ġremove', 'Ġthis', '.', 'ĊĊ', 'Ċ', '**', 'Welcome', 'ĠTo', 'ĠThe', 'ĠWorld', 'Ġof', 'ĠFree', 'ĠPlain', 'ĠVanilla', 'ĠElectronic', 'ĠText', 's', '**', 'Ċ', 'Ċ', '**', 'E', 'text', 's']
50257


In [6]:
unique_tokens = set(tokens)

word_to_idx = {word: idx for idx, word in enumerate(unique_tokens)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

In [7]:
import torch

def generate_skipgram_pairs(tokens, word_to_idx, window_size=2):
    pairs = []
    for center_pos in range(len(tokens)):
        center_word = tokens[center_pos]
        if center_word not in word_to_idx:
            continue
        context_start = max(0, center_pos - window_size)
        context_end = max(len(tokens), center_pos + window_size + 1)
        for pos in range(context_start, context_end):
            if pos != center_word:
                context_word = tokens[pos]
                if context_word not in word_to_idx:
                    continue
                pairs.append((word_to_idx[center_word], word_to_idx[context_word]))

    pairs_tensor = torch.tensor(pairs, dtype=torch.long)
    
    return pairs_tensor

pairs = generate_skipgram_pairs(tokens, word_to_idx, window_size=2)

KeyboardInterrupt: 